In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
0,Covid: FDA approves Pfizer vaccine for emergen...,https://www.bbc.com/news/world-us-canada-55265477,"Sat, 12 Dec 2020 15:50:00 GMT",2020-12-12 15:50:00+00:00
2,Source -- Carolina Panthers expected to fine p...,https://www.espn.com/nfl/story/_/id/30502520/s...,"Sat, 12 Dec 2020 15:43:32 GMT",2020-12-12 15:43:32+00:00
9,Latest on COVID-19 in MN: Deaths delivering a ...,https://www.mprnews.org/story/2020/12/12/lates...,"Sat, 12 Dec 2020 13:00:00 GMT",2020-12-12 13:00:00+00:00
4,Virologist Discusses FDA's COVID-19 Vaccine Au...,https://www.npr.org/2020/12/12/945788708/virol...,"Sat, 12 Dec 2020 12:58:00 GMT",2020-12-12 12:58:00+00:00
1,Rural Healthcare Systems Stretched To Limit Du...,https://www.npr.org/2020/12/12/945550903/rural...,"Sat, 12 Dec 2020 12:01:02 GMT",2020-12-12 12:01:02+00:00
8,'We're not winning this battle': Relentless CO...,https://www.usatoday.com/story/news/health/202...,"Sat, 12 Dec 2020 12:00:36 GMT",2020-12-12 12:00:36+00:00
3,This doctor has fought covid-19 in his patient...,https://www.washingtonpost.com/health/2020/12/...,"Sat, 12 Dec 2020 12:00:00 GMT",2020-12-12 12:00:00+00:00
5,Year in Review: COVID-19 hit post-acute care p...,https://www.modernhealthcare.com/post-acute-ca...,"Sat, 12 Dec 2020 06:00:00 GMT",2020-12-12 06:00:00+00:00
6,"Weekly COVID-19 Update - Dec. 11, 2020: Surge ...",https://news.delaware.gov/2020/12/11/weekly-co...,"Sat, 12 Dec 2020 01:43:41 GMT",2020-12-12 01:43:41+00:00
7,Baricitinib plus remdesivir shows promise for ...,https://www.nih.gov/news-events/news-releases/...,"Fri, 11 Dec 2020 18:04:11 GMT",2020-12-11 18:04:11+00:00
